# **Seminar 5 - Инструменты разработки**
*Naumov Anton (Any0019)*

*To contact me in telegram: @any0019*

## 1. HuggingFace

HuggingFace ( https://huggingface.co ) - один из ваших лучших друзей как ML-щиков

Это платформа для машинного обучения.

На платформе можно найти, а также добавлять и хостить модели, датасеты, api-ки

Также платформа имеет серьёзную и очень сильную python-библиотеку (вернее целое семейство библиотек) для ML.

### 1.0 python-библиотеки

У HuggingFace есть целый набор библиотек для ML

Для работы с моделями ( https://huggingface.co/docs/hub/models-libraries ), из самых важных:
- transformers - для работы с NLP
- diffusers - для работы с диффузионками
- PEFT - Parameter-Efficient Fine-Tuning (Lora)

Для работы с данными ( https://huggingface.co/docs/hub/datasets-libraries ), из самых важных:
- datasets - датасеты :)

В целом это даже близко не полный список ( https://github.com/huggingface ):
- evaluate ( https://github.com/huggingface/evaluate ) - разные метрики / бенчмарки
- accelerate ( https://github.com/huggingface/accelerate ) - multi-gpu обучения
- optimum ( https://github.com/huggingface/optimum ) - оптимизация инференса
- ...

sklearn в мире DL :D

In [ ]:
%pip install torch transformers datasets evaluate scikit-learn accelerate

### 1.1 Transformers - pipeline

Концепция pipeline-ов такова, что объединяются 3 вещи в одну конструкцию:
1. Пре-процессинг (токенизация, ...)
2. Модель
3. Пост-процессинг

https://huggingface.co/docs/transformers/index

In [ ]:
from transformers import pipeline

classifier = pipeline(
    task='sentiment-analysis',
    model="distilbert-base-uncased-finetuned-sst-2-english",
)

In [ ]:
print(classifier("This model is nice!"))

In [ ]:
print(classifier(
    [
        "What an awful thing...",
        "It's great in what it was designed for, but kinda awful that everything is done for me",
    ]
))

In [ ]:
classifier.model

In [ ]:
classifier.tokenizer

In [ ]:
?classifier.postprocess

In [ ]:
# mlm_model = pipeline('fill-mask', model="bert-base-uncased")
mlm_model = pipeline(task='fill-mask', model="bert-base-cased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

In [ ]:
?pipeline

In [ ]:
del classifier, mlm_model

Существует множество моделей под самые разные задачи - быстро найти любые модели: https://huggingface.co/models 

In [ ]:
import json

text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Задача: Создайте pipeline для Named Entity Recognition (NER) задачи, ищите модельки на хабе
#  - либо по тексту ner в названии
#  - либо по задаче Token Classification
ner_model = ...

named_entities = ner_model(text)
named_entities

In [ ]:
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

### 1.2 Transformers - model and tokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
]

# токенизация батча текстов. "pt" - [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key].shape, tokens_info[key], sep="\n", end="\n\n")

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

In [ ]:
text_for_analyse = "some random text for deeper analysis + weird word Rutherfordium"

In [ ]:
for key, value in tokenizer(text_for_analyse).items():
    print(key, value, sep="\n", end="\n\n")

In [ ]:
tokenizer.encode(text_for_analyse)

In [ ]:
tokenizer.decode(tokenizer.encode(text_for_analyse))

In [ ]:
tokenizer.batch_decode(tokenizer.encode(text_for_analyse))

In [ ]:
tokenizer.tokenize(text_for_analyse)

In [ ]:
(
    tokenizer.all_special_ids,
    tokenizer.all_special_tokens,
    tokenizer.all_special_tokens_extended,
    tokenizer.added_tokens_encoder,
    tokenizer.added_tokens_decoder,
)

In [ ]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
]

tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

# прямой проход через модель
with torch.no_grad():
    outputs = model(**tokens_info)

print(outputs)

In [ ]:
model.encoder.layer[-1].output

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
model.pooler

In [ ]:
outputs.pooler_output.shape

### 1.3 Datasets

https://huggingface.co/docs/datasets/index

In [ ]:
from datasets import load_dataset

ds = load_dataset("fancyzhx/yelp_polarity")

In [ ]:
ds

In [ ]:
ds["train"][0]

In [ ]:
ds["train"][0:5]["text"]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenize_function(ds["train"][0:5])

In [ ]:
tokenized_datasets = ds.map(tokenize_function, batched=True, batch_size=1000)

In [ ]:
tokenized_datasets["train"][0].keys()

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1024))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1024))

In [ ]:
from torch.utils.data import DataLoader
small_train_dataset.set_format(type="torch", columns=["input_ids", "label", "attention_mask"])
dataloader = DataLoader(small_train_dataset, batch_size=4)
res = next(iter(dataloader))

for key, value in res.items():
    print(key, value.shape, value, sep="\n", end="\n-------\n")

Умеет много чего
```python
ds.rename_column("text", "unsplit_text")  # переименовывать колонки
ds.cast_column("image", Image(mode="RGB"))  # приводить отдельные колонки к нужному виду
dataset.with_transform(transforms)  # аугументации на бегу
...
```

### 1.4 Evaluate

https://huggingface.co/docs/evaluate/index

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
metric.compute(predictions=[1, 2, 3, 4], references=[1, 1, 1, 4])

In [ ]:
metric.compute(predictions=[1, 2, 3, 4], references=[4, 3, 2, 1])

In [ ]:
metric.compute(predictions=[1, 2, 3, 4], references=[1, 2, 3, 4])

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 1.5 Transformers - Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

?TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_model",
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    # lr_scheduler_kwargs={},
    # warmup_ratio=0.03125,
    # warmup_steps=10,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    log_level="error",
    # logging_dir="output_dir/runs/CURRENT_DATETIME_HOSTNAME"  # логи для tensorboard (default)
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch",
    # save_steps=1,
    save_total_limit=2,
    save_safetensors=True,  # safetensors вместо torch.save / torch.load
    save_only_model=False,  # сохраняем optimizer, shceduler, rng, ...
    use_cpu=False,
    seed=42,
    # bf16=True,  # использовать bf16 вместо fp32
    eval_strategy="epoch",
    # eval_steps=32,
    disable_tqdm=False,
    load_best_model_at_end=False,
    label_smoothing_factor=0.,
    optim="adamw_torch",
    # optim_args=...,
    # resume_from_checkpoint=...,
    # auto_find_batch_size=...,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
texts = [
    "This was not a good movie!",
    "What an awesome place!",
    "ewww",
]

tokens_info = tokenizer(
    texts,
    padding=True,
    truncation=True,
    return_tensors="pt",
)

model.eval()
model.cpu()
with torch.no_grad():
    out = model(**tokens_info)
    probs = torch.nn.functional.softmax(out.logits, dim=-1)
    for text, prob in zip(texts, probs.tolist()):
        print(
            f"Text: `{text}`\nPrediction (prob): "
            f"positive={round(prob[0], 3)} ; "
            f"negative={round(prob[1], 3)}",
            end="\n\n"
        )

## 2. StreamLit

StreamLit - простая библиотека для построения интерактивных веб-приложений

In [1]:
%pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


```bash
streamlit hello  # демо с кодом от самого streamlit
```

Приложения через streamlit строятся построчно, а не от макета

Основные принципы:
1. Используй скрипты на Python. Построчно создавайте и расширяйте приложения Streamlit.
2. Рассматривай виджеты как переменные. Виджеты - это элементы ввода, которые позволяют пользователям взаимодействовать с приложениями Streamlit. Они представлены в виде основных текстовых полей ввода, флажков, ползунков и т.д.
3. Повторно используй данные и вычисления. Исторически данные и вычисления кэшировались с помощью @st.cache декоратора. Это экономит вычислительное время при внесении изменений в приложение. Это может происходить сотни раз, если ты активно редактируешь приложение! В версии 0.89.0 Streamlit запустил два новых примитива (st.experimental_memo и st.experimental_singleton), что позволило значительно повысить скорость работы по сравнению с @st.cache.

In [ ]:
import streamlit as st

st.__version__

Пайплайн приложения
1. Создаётся и заполняется файл `app.py` (default, можете свой)
2. `streamlit run app.py`
3. Done!

### 2.1. Текст

In [ ]:
import streamlit as st

st.title("This is a title")
st.header("This is a header")
st.subheader("This is a subheader")
st.text("This is a text")
st.markdown("# This is a markdown header 1")
st.markdown("## This is a markdown header 2")
st.markdown("### This is a markdown header 3")
st.markdown("This is a markdown: *bold* **italic** `inline code` ~strikethrough~")
st.markdown("""This is a code block with syntax highlighting
```python
print("Hello world!")
```
""")
st.html(
    "image from url example with html: "
    "<img src='https://www.wallpaperflare.com/static/450/825/286/kitten-cute-animals-grass-5k-wallpaper.jpg' width=400px>",
)


st.write("Text with write")
st.write(range(10))

### 2.2. Логирование

In [ ]:
st.success("Success")
st.info("Information")
st.warning("Warning")
st.error("Error")
exp = ZeroDivisionError("Trying to divide by Zero")
st.exception(exp)

### 2.3. Объекты

In [ ]:
from urllib import request
request.urlretrieve(
    "http://craphound.com/images/1006884_2adf8fc7.jpg",
    "image_example.jpg",
)

from PIL import Image
img = Image.open("image_example.jpg")
img

In [ ]:
# картинка (без html - из переменной)
st.image(img, width=200)

# чекбокс
if st.checkbox("Show/Hide"):
    st.text("Showing the widget")
else:
    st.warning("Not showing what is inside")

# выбор опции кружочками
status = st.radio("Select Gender: ", ('Male', 'Female'))
if (status == 'Male'):
    st.success("Male")
else:
    st.success("Female")

# выбор опции выпадающим меню
hobby = st.selectbox(
    "Hobbies: ",
    ['Dancing', 'Reading', 'Sports'],
)
st.write("Your hobby is: ", hobby)

# выбор нескольких опций
hobbies = st.multiselect(
    "Hobbies: ",
    ['Dancing', 'Reading', 'Sports'],
)
st.write("You selected", len(hobbies), 'hobbies')

# кнопка без функционала
st.button("Click me for no reason")

# кнопка, показывающая текст, когда нажата
if(st.button("Click me")):
    st.text("You did it, you clicked me!!!")

# текстовый input: label - название, value - что написано по дефолту
name = st.text_input(label="Enter Your name", value="Type Here ...")
if(st.button('Submit')):
    result = name.title()
    st.success(result)

# слайдер
level = st.slider("Select the level", 1, 5)
st.text('Selected: {}'.format(level))

### 2.4. Сложные действия

```python
# Переменная общая на rerun - способ шейрить информацию между изменениями
st.session_state  # kinda Dict[str, Any]

# Инициализация
if 'key' not in st.session_state:
    st.session_state['key'] = 'value'

# Можно также обращаться по атрибутам, а не ключам
if 'key' not in st.session_state:
    st.session_state.key = 'value'
```

In [ ]:
# инициализируем переменные
st.session_state.key1 = 'value1'     # Attribute API
st.session_state['key2'] = 'value2'  # Dictionary like API

# посмотреть что в st.session_state
st.write(st.session_state)

# magic
st.session_state

# ошибка если неправильный ключ
st.write(st.session_state['missing_key'])

In [ ]:
# key - позволяет указать в какое поле session_state записать объект
st.text_input("Please input something", key="my input")
st.session_state

### 2.5. Кэширование

Для кэширования есть 2 декоратора

```python
@st.cache_data      # для данных - сериализация выходов с ключами входов
@st.cache_resource  # для моделей / ресурсов - несериализуемые объекты, которые не хочется загружать несколько раз
```

In [ ]:
import streamlit as st
import pandas as pd

@st.cache_data  # кэширование
def load_data(url):
    df = pd.read_csv(url)  # скачивание датасета
    return df

df = load_data("https://github.com/plotly/datasets/raw/master/uber-rides-data1.csv")
st.dataframe(df)

st.button("Rerun")

In [ ]:
import streamlit as st
from transformers import pipeline

@st.cache_resource  # кэширование
def load_model():
    return pipeline("sentiment-analysis")  # скачивание модели

model = load_model()

query = st.text_input("Your query", value="I love Streamlit! 🎈")
if query:
    result = model(query)[0]  # классифицируем
    st.write(query)
    st.write(result)

## 3. HF + StreamLit

Можно поднять тестовую streamlit api прямо на hugging face

1. https://huggingface.co/
2. New space - Streamlit
3. Делаем `app.py` и `requirements.txt`
4. Собирается докер образ - появляется app (публично доступен)
5. \* немного хулиганства - можно достать даже iframe из hf